In [1]:
from API import API
api = API()

Implicit Grant Flow

Logging in...


Implicit Grant Flow failed. Trying Client Credentials Flow.

OK
Client Credential Flow. If you wish to alter user playlists, look up how to run selenium with the Firefox self.driver.


In [3]:
print(api.ACCESS_TOKEN)
print(api.REFRESH_TOKEN)

BQDmE1wdHPHv95ghrthK14QadPMsuTw9Yr8xDk4XT1LmZp4BGYq7A_uJZvZtvKEB-7sBtfSaCXN1BM9GqUuFV70ASyQ5HdZNo701GMmTEIYTSqR1yQgD
None


In [ ]:
# TODO check whether token is expired
api.getPlaylists()

In [3]:
response = api.ClientCredentialsFlow()
response

OK


<Response [200]>

In [11]:
import json
json.loads(response.text)['expires_in']

3600

In [14]:
import time
time_og = time.time()
time_og

1677585282.0268934

In [15]:
time.ctime(time_og)

'Tue Feb 28 12:54:42 2023'

In [16]:
time.ctime(time_og + 3600)

'Tue Feb 28 13:54:42 2023'

In [5]:
time.ctime(api.expiration)

'Tue Feb 28 13:56:39 2023'